In [32]:
import dt_apriltags as apriltag
import cv2
import annotation_utils as utils

import numpy as np
from scipy.spatial.transform import Rotation as R

import pyrealsense2 as rs
import urx
np.set_printoptions(suppress=True)

# dist = np.array([[-0.04797802,  0.04744357,  0.00017416,  0.00067967, -0.00408397]])
# dist = np.array([[-0.,  0.,  0.,  0., -0.]])
mtx = np.array([[648.09029639, 0., 648.06462963], [0., 639.7544391, 359.74013262], [0., 0., 1.]])

camera_params = [648.0, 648.0, 639.0646296262861, 359.7401326185789]
dist = np.array([[0.028,  -0.0844,  0.00069588,  0.00022173, -0.12084083]])
corn_coords = 56* 0.3
#(0, 0,0) 
corners_3D = np.array(
[
 (corn_coords, corn_coords,0),
 (corn_coords, -corn_coords,0),
 (-corn_coords, -corn_coords,0),
 (-corn_coords,corn_coords, 0)], dtype=float)

keypoints, anchors = utils.get_points_from_CAD()
# anchors[:,2] +=1

Rz_180 = R.from_euler('xyz',[0,0,np.pi])
keypoints = np.hstack((keypoints,np.zeros((keypoints.shape[0],1))))
keypoints = Rz_180.apply(keypoints)
anchors = Rz_180.apply(anchors)

keypoints = keypoints[:,:2]

keypoints = np.hstack((keypoints,np.ones((keypoints.shape[0],1))))
print(keypoints)
print(anchors)



# [[648.70490716   0.         639.83974283]
#  [  0.         648.98850876 359.4889819 ]
#  [  0.           0.           1.        ]] [ 0.02779783 -0.08445151  0.00069588 -0.00022173  0.12084083]

[[ -6.143 -37.51    1.   ]
 [  6.143 -37.51    1.   ]
 [-27.213 -28.666   1.   ]
 [ 27.213 -28.666   1.   ]
 [-13.161 -26.86    1.   ]
 [ 13.161 -26.86    1.   ]
 [-27.707 -19.26    1.   ]
 [ 27.707 -19.26    1.   ]
 [-38.253  -5.53    1.   ]
 [ 38.253  -5.53    1.   ]
 [-19.9    27.273   1.   ]
 [ 19.9    27.273   1.   ]
 [-20.4    29.39    1.   ]
 [ 20.4    29.39    1.   ]]
[[  0.     0.     2.25]
 [ -8.   -11.2    2.25]
 [  8.   -11.2    2.25]
 [-16.     0.     2.25]
 [ 16.    -0.     2.25]
 [ -8.    13.86   2.25]
 [  8.    13.86   2.25]]


In [33]:
def solve_pnp(obj_points, imagePoints,mtx, dist,reprojectionError ):
    success, rvec, tvec,inliers  = cv2.solvePnPRansac(obj_points, np.array([imagePoints]), mtx, dist,reprojectionError = reprojectionError)
        
    if not success:
        print('not success in PnP')
        return
    return tvec, rvec,inliers


def print_error_rotation(tf_error):
    vec = R.from_matrix(tf_error[:3,:3]).as_rotvec() 
    print("vector in marker", vec/np.linalg.norm(vec))
    print("angles, deg", np.linalg.norm(vec)*57)

def calculate_corrected_kts(tf_error,keypoints):
    keypoints3d_homog = np.hstack((
        keypoints,
        np.ones((keypoints.shape[0],1))))

    keypoints_corrected = (tf_error @ keypoints3d_homog.T).T
    return keypoints_corrected

def get_marker_pose(img_with_marker):
    res = utils.detect_apriltag(img_with_marker, camera_params)

    if res is None:
        return None,None
    corners_2d = res[0].corners
    center_2d = res[0].center
    
    if False:
        img_with_marker = cv2.imread(img_with_marker)
        for p in corners_2d.reshape(-1,2):
            kp = int(p[0]), int(p[1])
            cv2.circle(img_with_marker, kp, 2, (0, 0, 255), -1)
        cv2.imshow("Image", img_with_marker)
        cv2.waitKey(0)
        
    marker_pts_2d = np.vstack((center_2d, corners_2d))

    tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,corners_2d,mtx, dist, 0.5)
    
    return tvec_m,rvec_m

def get_socket_points(img, points_array, tvec, rvec, 
                      show=True):
    if isinstance(img, str):
        img = cv2.imread(img)
    img_points = None
    img_points, _ = cv2.projectPoints(points_array, rvec, tvec, mtx, dist)
    img_points = img_points.reshape(-1,2)

    if show:
        for p in img_points.reshape(-1,2):
            kp = int(p[0]), int(p[1])
            cv2.circle(img, kp, 2, (0, 0, 255), -1)
        cv2.imshow("Image", img)
        cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img_points

In [53]:
def solve_correction(img_with_marker, anchor_gt_2D):
    def apply_correction():
        result = tf_s_in_c.copy()
        z_from = tf_s_in_c[:3,2]
        z_to = tf_m_in_cam[:3,2]
        angle = np.arccos(z_from@z_to)
        rot_ax = np.cross(z_from,z_to)

        rot_vec = rot_ax /np.linalg.norm(rot_ax) * angle
        rot_mtx = R.from_rotvec(rot_vec).as_matrix()
        # print(rot_mtx @ z_from -z_to )

        result[:3,:3]  = rot_mtx @ tf_s_in_c[:3,:3]
        # correction for OZ
        # d_cam = - (tf_s_in_c[:3,3] - tf_m_in_cam[:3,3])
        d_cam = - (tf_s_in_c[:3,3] - tf_m_in_cam[:3,3])
        z_proj = (d_cam) @ z_to
        corr_z = np.eye(4)
        corr_z[2,3] = z_proj

        return  result @ corr_z

    res = utils.detect_apriltag(img_with_marker, camera_params)

    corners_2d = res[0].corners
    center_2d = res[0].center

    # marker_pts_2d = np.vstack((center_2d, corners_2d))
    # tvec_m,rvec_m,inliers_m = solve_pnp(corners_3D,corners_2d,mtx, dist, 0.5)
    tvec_m = res[0].pose_t
    # * R.from_euler('xyz',[0,0,np.pi])
    rvec_m = (R.from_matrix(res[0].pose_R)).as_rotvec()
    # print("inliers_m\n",inliers_m.T)

    tf_m_in_cam = np.eye(4)
    tf_m_in_cam[:3,:3] = R.from_rotvec(rvec_m.flatten()).as_matrix()
    tf_m_in_cam[:3,3] = tvec_m.flatten()

    tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchor_gt_2D,mtx, dist,1.0)
    print(R.from_rotvec(rvec_gt.flatten()).as_matrix())
    # print("inliers_gt\n",inliers_gt.T)

    tf_s_in_c = np.eye(4)
    tf_s_in_c[:3,:3] = R.from_rotvec(rvec_gt.flatten()).as_matrix()
    tf_s_in_c[:3,3] = tvec_gt.flatten()
    tf_s_in_c_corrected = apply_correction()
    # tf_err_in_m = np.linalg.inv(tf_m_in_cam) @ tf_s_in_c_corrected
    tf_err_in_m = np.linalg.inv(tf_m_in_cam) @ tf_s_in_c_corrected
    tf_err_in_m[:3,3] *=-1
    return tf_err_in_m

In [47]:
# tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchor_gt_2D,mtx, dist,1.0)
# print(R.from_rotvec(rvec_gt.flatten()).as_matrix())
# cv2.destroyAllWindows()
# img_with_marker = cv2.imread(img_with_marker_name)

# tvec_m,rvec_m = get_marker_pose(img_with_marker)

# img_without_marker = cv2.imread(img_without_marker_name)
# get_socket_points(img_without_marker, corr_kpts[:,:3], tvec_m,rvec_m)
# get_socket_points(img_without_marker, keypoints, tvec_m,rvec_m)


NameError: name 'anchor_gt_2D' is not defined

In [59]:
# averaging the corrections
import json

def visualize_kpts(img, points_array,color = (0, 0, 255),show_img = False):
    # if isinstance(img, str):
    img = cv2.imread(img)
    
    for p in points_array.reshape(-1,2):
        kp = int(p[0]), int(p[1])
        cv2.circle(img, kp, 1, color, -1)
    
    if show_img:
        cv2.imshow("Image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return img
    
path_to_json = "data_05/calibration/json/captures_000.json"

calib_data = []
with open(path_to_json, "r") as read_file:
    calib_data = json.load(read_file)

path_to_json = "data_05/with_cover/json/captures_000.json"

data = []
with open(path_to_json, "r") as read_file:
    data = json.load(read_file)
id = 8
img_path = 'data_05/calibration/rgb/' + calib_data['captures'][id]["filename"].split('/')[1]
# print(img_path)
res = utils.detect_apriltag(img_path, camera_params)
tvec_m = res[0].pose_t
# print(res[0].pose_R)
# * R.from_euler('xyz',[0,0,np.pi])
rvec_m = (R.from_matrix(res[0].pose_R) ).as_rotvec()

kpts_gt = calib_data['captures'][id]["annotations"][0]['values'][0]['keypoints'][:-7] 
kpts_gt = [[anch['x'],anch['y']] for anch in kpts_gt]

anchors_2d = calib_data['captures'][id]["annotations"][0]['values'][0]['keypoints'][-7:] 
anchors_2d = [[anch['x'],anch['y']] for anch in anchors_2d]
# print(anchors)
# print(len(kpts_gt))
# print(kpts_gt)
visualize_kpts(img_path,np.array(kpts_gt),(0,0,255),True)
tvec_gt,rvec_gt,inliers_gt = solve_pnp(anchors,anchors_2d,mtx, dist,1.0)
# # print(inliers_gt)
# print(tvec_gt-tvec_m)
ang_diff = R.from_rotvec(rvec_m.flatten()).as_matrix()@ R.from_rotvec(rvec_gt.flatten()).as_matrix().T
# print(np.linalg.norm(R.from_matrix(ang_diff).as_rotvec())*57)
# kpts_list = []
tf_err_in_m = solve_correction(img_path, anchors_2d)

corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)
# # kpts_list.append(corr_kpts)
corr_kpts_new = np.array(corr_kpts[:,:3],dtype=float)

img_with_marker = 'data_05/with_cover/rgb/' + data['captures'][id]["filename"].split('/')[1]
# img_with_marker = cv2.imread(img_path)

res = utils.detect_apriltag(img_with_marker, camera_params)
tvec_m = res[0].pose_t
# * R.from_euler('xyz',[0,0,np.pi])
rvec_m = (R.from_matrix(res[0].pose_R) ).as_rotvec()
# tvec_m,rvec_m = get_marker_pose(img_with_marker)

img_without_marker = 'data_05/no_cover/rgb/' + data['captures'][id]["filename"].split('/')[1]
# # img_without_marker = cv2.imread(img_without_marker_name)
ktps_achieved2 = get_socket_points(img_without_marker,keypoints, tvec_m,rvec_m)
ktps_achieved = get_socket_points(img_without_marker,corr_kpts_new, tvec_m,rvec_m)

kpts_gt = data['captures'][id]["annotations"][0]['values'][0]['keypoints'][:-7] 
kpts_gt = [[anch['x'],anch['y']] for anch in kpts_gt]
# # print(kpts_gt)
print(ktps_achieved -kpts_gt)

reading img from file system
reading img from file system
[[ 0.9998177   0.00261538 -0.01891373]
 [-0.00237735  0.99991783  0.01259652]
 [ 0.01894512 -0.01254926  0.99974177]]
reading img from file system
[[0.55293047 2.26936766]
 [0.68726281 2.19643065]
 [0.31317674 2.32892459]
 [1.11581321 1.8686585 ]
 [0.47921318 2.23303406]
 [0.73746843 2.07824675]
 [0.31357579 2.23734982]
 [0.88834136 1.90774537]
 [0.20238013 2.09824175]
 [0.97145774 1.63378181]
 [0.3554974  1.69132813]
 [0.7313647  1.43607137]
 [0.34714158 1.66305074]
 [0.73384322 1.3996426 ]]


In [60]:
print(ktps_achieved2 -kpts_gt)

[[-0.36927921  2.93645163]
 [-0.21669338  1.78576361]
 [ 0.13594002  4.79993595]
 [ 1.0622954  -0.50509362]
 [ 0.54845363  3.46813118]
 [ 0.87199691  0.96359098]
 [ 1.02486072  4.73921431]
 [ 1.77578946 -0.59974898]
 [ 2.13964032  5.54315258]
 [ 3.25356852 -1.99580919]
 [ 5.7626279   3.50680446]
 [ 6.46003614 -0.63022808]
 [ 5.96818174  3.53181669]
 [ 6.69421882 -0.72479348]]


In [67]:
np.array(ktps_achieved2 -kpts_gt).var()

6.122763203849719

In [48]:
anchors_2d

[[714.2413, 294.794525],
 [671.512939, 235.495758],
 [756.9795, 235.001129],
 [628.9895, 295.0524],
 [800.1727, 294.534546],
 [671.5533, 368.7039],
 [757.297852, 368.7386]]

In [ ]:
# corners_3D
import importlib

# importlib.reload(utils)

path_to_json = "generated_photos/json/captures_000.json"
with open(path_to_json, "r") as read_file:
    data = json.load(read_file)
N = 100
total3d = [corners_3D for i in range(N)]
# np.repeat(corners_3D, 4, axis = 0).reshape()
corners2d = []
for id in range(2,2+N):
    img_path = 'generated_photos/rgb/' + data['captures'][id]["filename"].split('/')[1]
    gray = cv2.imread(img_path)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    shape = gray.shape[::-1]
    res = utils.detect_apriltag(img_path, camera_params)
    corners2d.append(res[0].corners)
total3d = np.array(total3d,dtype = np.float32)
corners2d = np.array(corners2d,dtype = np.float32)
print(total3d.shape)
print(corners2d.shape)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera (total3d,corners2d,shape,None,np.zeros(5))
print(mtx, dist)
# calibrateCamera 
# initCameraMatrix2D


reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img from file system
reading img fr

In [ ]:
anchors_list = [
    [[699, 421],[682, 397],[717, 397],[665, 421],[735, 421],[682, 451],[717, 451]],
    [[608, 418],[594, 398],[623, 398],[ 578, 418],[368, 418],[593, 444],[623, 444]],
    [[610, 417],[597, 399],[623, 399],[584, 417],[636, 417],[597,439],[623, 439]]
]
pics_list = [335, 559, 784]

kpts_list = []
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    anchor_gt_2D = np.array(anchors_list[i], dtype = float)
    img_with_marker = cv2.imread(img_with_marker_name)
    img_without_marker = cv2.imread(img_without_marker_name)
    
    tf_err_in_m = solve_correction(img_with_marker, anchor_gt_2D)
    corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)
    kpts_list.append(corr_kpts)

kpts_list = np.array(kpts_list)
kpts_list.var
corr_kpts_mean = np.mean(kpts_list, axis = 0, dtype=np.float32)
corr_kpts_new = np.array(corr_kpts_mean[:,:3],dtype=float)
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)

    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    
    img_without_marker = cv2.imread(img_without_marker_name)
    get_socket_points(img_without_marker,corr_kpts_new , tvec_m,rvec_m)

In [ ]:
# averaging the corrections

anchors_list = [
    [[699, 421],[682, 397],[717, 397],[665, 421],[735, 421],[682, 451],[717, 451]],
    [[608, 418],[594, 398],[623, 398],[ 578, 418],[368, 418],[593, 444],[623, 444]],
    [[610, 417],[597, 399],[623, 399],[584, 417],[636, 417],[597,439],[623, 439]]
]
pics_list = [335, 559, 784]

kpts_list = []
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    anchor_gt_2D = np.array(anchors_list[i], dtype = float)
    img_with_marker = cv2.imread(img_with_marker_name)
    img_without_marker = cv2.imread(img_without_marker_name)
    
    tf_err_in_m = solve_correction(img_with_marker, anchor_gt_2D)
    corr_kpts = calculate_corrected_kts(tf_err_in_m,keypoints)
    kpts_list.append(corr_kpts)

kpts_list = np.array(kpts_list)
kpts_list.var
corr_kpts_mean = np.mean(kpts_list, axis = 0, dtype=np.float32)
corr_kpts_new = np.array(corr_kpts_mean[:,:3],dtype=float)
for i in range(3):
    img_with_marker_name = "data_for_corr_new/" + str(pics_list[i]) + ' (2).jpg'
    img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)

    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    
    img_without_marker = cv2.imread(img_without_marker_name)
    get_socket_points(img_without_marker,corr_kpts_new , tvec_m,rvec_m)

In [ ]:
Tc = np.eye(4)
Tc[:3,:3] = R.from_euler('xyz', [30,10,-10], degrees = True).as_matrix()
Tc[:3,3] = np.array([100,30,50])

T = np.eye(4)
T[:3,:3] = R.from_euler('xyz', [-30,54,-5], degrees = True).as_matrix()
T[:3,3] = np.array([18,-33,10])
point = np.zeros(4)
point[-1] = 1
point = T @ point
print(point)
prox = Tc @ point.T @ np.linalg.inv(Tc)
# prox /= prox[-1]
print(prox)


[ 18. -33.  10.   1.]
[   107.96505922    -44.11359511     11.33170204 -10038.68317055]


In [ ]:
data = {}
photos_to_ignore = [i for i in range(18,27)]
processed_data = 0
for i in range(10):
    if i in photos_to_ignore:
        continue
    img_with_marker_name = "close_photos_marker/" + str(i) + '.jpg'
    img_without_marker_name = "close_photos_no_marker_normal/" + str(i) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)

    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    print(tvec_m,rvec_m)
    if tvec_m is None:
        continue
    img_without_marker = cv2.imread(img_without_marker_name)
    data[i] = get_socket_points(img_without_marker,corr_kpts_new , tvec_m,rvec_m, show=False).tolist()
    
    processed_data += 1
    
with open("ground_truth_keypoints.json", "w") as write_file:
    json.dump(data, write_file)

[[-104.5055155 ]
 [ -51.2867595 ]
 [ 166.23368801]] [[-0.11871177]
 [ 0.11755822]
 [ 3.13252847]]
[[-77.81140108]
 [-51.10031104]
 [172.98982816]] [[0.02117258]
 [0.13627275]
 [3.13210956]]
[[-50.57850197]
 [-50.83974876]
 [176.80210345]] [[0.16179013]
 [0.14707179]
 [3.12633491]]
apriltag not found
None None
apriltag not found
None None
apriltag not found
None None
apriltag not found
None None
apriltag not found
None None
apriltag not found
None None
[[-104.62337024]
 [ -16.20614061]
 [ 164.98202395]] [[-0.12053094]
 [ 0.11408218]
 [ 3.13436299]]


In [ ]:
import json
data = {}
# "10" +

for i in range(100):
    img_with_marker_name = "100_photos/" + "4_" + str(i)+'.png'
    # print(img_with_marker_name)
    # img_without_marker_name = "data_for_corr_new/" + str(pics_list[i]) + '.jpg'
    img_with_marker = cv2.imread(img_with_marker_name)
    img_with_marker = cv2.rotate(img_with_marker, cv2.ROTATE_90_COUNTERCLOCKWISE)
    tvec_m,rvec_m = get_marker_pose(img_with_marker)
    if tvec_m is None:
        raise

    # img_without_marker = cv2.imread(img_without_marker_name)
    data[i] = get_socket_points(img_with_marker,corr_kpts_new , tvec_m,rvec_m, show=False).tolist()

    
with open("4.json", "w") as write_file:
    json.dump(data, write_file)


In [ ]:
# print(data.items())

In [ ]:
keypoints_new = (tf_s_in_c @ anchor_kpts.T).T
print(keypoints_new)
print(keypoints.shape)
keypoints = np.hstack((
    keypoints,
    np.zeros((keypoints.shape[0],1)),
    np.ones((keypoints.shape[0],1))))


keypoints_corrected = (tf_err_in_m @ keypoints.T).T

print(keypoints_corrected)

NameError: name 'tf_s_in_c' is not defined

In [ ]:
# old data for correction
anchors_list = [
    [[654, 358], [630,325],[680,324],[606,359],[703,358],[631,399],[679,399]],
    [[652, 355],[630, 326],[673, 325],[609,356],[695, 356],[631, 393],[674, 393]],
    [[648, 355],[630, 329],[670,328],[611, 355],[688, 355],[630,389],[669, 389]],
    [[648,355],[630,331],[666, 331],[613, 355],[684,355],[630,386],[666, 386]],
    [[647, 355],[630,332],[663, 332],[614, 355],[679,355],[630,383],[663,383]]
]
pics_list = [44, 108, 172,236,300]
img_with_marker_name = "data_for_correction/" + str(pics_list[i]) + ' (2).jpg'
img_without_marker_name = "data_for_correction/" + str(pics_list[i]) + '.jpg'